In [1]:
import datetime as dt

import pandas as pd
import numpy as np
from lenskit.algorithms import item_knn, user_knn
from sklearn.model_selection import ParameterGrid

from recsys4daos.datasets import to_lenskit
from recsys4daos.model_selection import cvtt_open, explore_hparams
from recsys4daos.evaluation import test_with_hparams_lenskit

import paths

/home/daviddavo/recsys4daos/src/recsys4daos/model_selection.py:12: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


# Parameters

In [2]:
# Dataset config
ORG_NAME = 'Decentraland'
SPLITS_FREQ = 'W-THU'  # Split weekly
LAST_FOLDS = 20  # Use just last 10 splits
SPLITS_NORMALIZE = True

# Evaluation
K_RECOMMENDATIONS: list[int] = [1,3,5,10,15,100]
SPLITS_FREQ: str = 'W-THU' # split weekly
SPLITS_NORMALIZE = True # Wether or not to move everything to 00:00
LAST_FOLDS = 10 # Use just last 10 splits
LAST_FOLD_DATE_STR: str = None

# Search space config
WINDOW_SIZES = ['7d', '14d', '21d', '30d', '60d', '90d', '10YE']
ITEMKNN_Ks = [1,2,3,4,5,6,7,8,9,10,15]

OPTIM_METRIC = 'map@10'

In [3]:
# Parameters
EXECUTION_ID = "2024-09-04T10:00"
ORG_NAME = "Genesis Alpha"
SPLITS_FREQ = "7d"
LAST_FOLDS = 10
SPLITS_NORMALIZE = True
LAST_FOLD_DATE_STR = "2019-12-04"


# Load the dataset

In [4]:
dfp = paths.load_proposals(ORG_NAME)
dfv = paths.load_votes(ORG_NAME)

print(dfp.info())
print(dfv.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 387 entries, 0 to 386
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   id                 387 non-null    object        
 1   author             387 non-null    object        
 2   date               387 non-null    datetime64[us]
 3   start              387 non-null    datetime64[us]
 4   end                387 non-null    datetime64[us]
 5   platform_proposal  387 non-null    object        
dtypes: datetime64[us](3), object(3)
memory usage: 18.3+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2203 entries, 0 to 2202
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   id        2203 non-null   object        
 1   proposal  2203 non-null   object        
 2   voter     2203 non-null   object        
 3   date      2203 non-null   datetime64[us]
dtypes: d

In [5]:
df = to_lenskit(dfv)
df

,user,item,timestamp,rating
0,0x144c4e5027b69f7798b2b162d924bcae5c149f15,5542b602-fae5-5894-8139-1bbc3f6fa3f9,2019-04-10 19:38:39,1
1,0xf3215bf9ad166213e2588baf81d7e932fda99d9e,5542b602-fae5-5894-8139-1bbc3f6fa3f9,2019-04-10 21:25:42,1
2,0x24597c5c68687e816ffc0c69e064cb70bb62a9cd,5542b602-fae5-5894-8139-1bbc3f6fa3f9,2019-04-10 22:06:20,1
3,0x6ac0a885ed84f4a2d062c60fb7daaf504fc8c47f,5542b602-fae5-5894-8139-1bbc3f6fa3f9,2019-04-11 07:24:41,1
4,0xc4d9d1a93068d311ab18e988244123430eb4f1cd,c56c81ad-7bf9-5aa7-bbbe-b643317cfc6d,2019-04-11 08:23:09,1
...,...,...,...,...
2198,0x0ebbda43014a6e6c32657a8db79376738d549b2f,c6ee2096-b2dc-51ee-bc1f-b96653e2c0bf,2020-04-29 05:44:45,1
2199,0x0ebbda43014a6e6c32657a8db79376738d549b2f,052958a4-180c-54ad-a4ba-dfc0c210dd3b,2020-05-02 06:32:56,1
2200,0x4171160db0e7e2c75a4973b7523b437c010dd9d4,c6ee2096-b2dc-51ee-bc1f-b96653e2c0bf,2020-05-07 14:36:22,1
2201,0x00234af289964cc53cae46452ddac6253fc3ec3a,d5234f86-22bc-5f5e-9b91-eb91719d4bfb,2020-12-12 18:37:20,1


## Split in folds

In [6]:
all_folds = { f.end:f for f in cvtt_open(
    df, SPLITS_FREQ, dfp.reset_index(), remove_not_in_train_col='item', col_item='item', last_fold=LAST_FOLD_DATE_STR,
)}
last_folds_idx = list(all_folds.keys())[-LAST_FOLDS:]
last_folds_idx

[Timestamp('2019-10-02 00:00:00'),
 Timestamp('2019-10-09 00:00:00'),
 Timestamp('2019-10-16 00:00:00'),
 Timestamp('2019-10-23 00:00:00'),
 Timestamp('2019-10-30 00:00:00'),
 Timestamp('2019-11-06 00:00:00'),
 Timestamp('2019-11-13 00:00:00'),
 Timestamp('2019-11-20 00:00:00'),
 Timestamp('2019-11-27 00:00:00'),
 Timestamp('2019-12-04 00:00:00')]

# Item-based KNN

In [7]:
def testHParamsItemKNN(fold, k: int, window_size=None):
    # Create algorithm
    algo = item_knn.ItemItem(
        feedback='implicit', # VERY IMPORTANT
        min_sim=0,
        # min_nbrs=0,
        nnbrs=k,  # the maximum number of neighbors for scoring each item (None for unlimited)
    )
    return test_with_hparams_lenskit(algo, all_folds[fold], K_RECOMMENDATIONS, window_size)

pd.Series(testHParamsItemKNN(last_folds_idx[-1],5,'14d'))

/home/daviddavo/recsys4daos/.direnv/python-3.11/lib/python3.11/site-packages/numba/np/ufunc/parallel.py:371: NumbaWarning: The TBB threading layer requires TBB version 2021 update 6 or later i.e., TBB_INTERFACE_VERSION >= 12060. Found TBB_INTERFACE_VERSION = 12050. The TBB threading layer is disabled.
  warnings.warn(problem)
Numba is using threading layer omp - consider TBB


found 1 potential runtime problems - see https://boi.st/lkpy-perf


/home/daviddavo/recsys4daos/.direnv/python-3.11/lib/python3.11/site-packages/lenskit/algorithms/item_knn.py:119: NumbaTypeSafetyWarning: unsafe cast from uint64 to int64. Precision may be lost.
  b = blocks[bi]


fold_t             2019-12-04 00:00:00
time_train                    5.553864
time_rec                      0.019126
open_proposals                       9
min_recs                             5
avg_recs                        8.0625
precision@1                     0.1875
precision@3                     0.1875
precision@5                     0.1875
precision@10                    0.1625
precision@15                  0.108333
precision@100                  0.01625
ndcg@1                        0.166667
ndcg@3                        0.260088
ndcg@5                        0.335998
ndcg@10                       0.479624
ndcg@15                       0.479624
ndcg@100                      0.479624
map@1                           0.1875
map@3                         0.232639
map@5                          0.29125
map@10                        0.378348
map@15                        0.378348
map@100                       0.378348
recall@1                      0.114583
recall@3                 

## Exploring hparams

In [8]:
results = explore_hparams(
    testHParamsItemKNN, 
    ParameterGrid({
    'fold': last_folds_idx,
    'k': ITEMKNN_Ks, # ,20,25,30,35,40,45,50,60,70,80,90,100],
    'window_size': WINDOW_SIZES,
    }),
    paths.hparams_progress('itemknn', ORG_NAME, SPLITS_FREQ, SPLITS_NORMALIZE),
)
mdfi = pd.DataFrame(results)
mdfi

Restored checkpoint from ../.cache/Genesis Alpha/hparams-itemknn_7d_normalize.pkl with 770 results


  0%|          | 0/770 [00:00<?, ?it/s]

,fold,k,window_size,fold_t,time_train,time_rec,open_proposals,min_recs,avg_recs,precision@1,...,recall@10,recall@15,recall@100,r-precision@1,r-precision@3,r-precision@5,r-precision@10,r-precision@15,r-precision@100,time_eval
0,2019-10-02,1,7d,2019-10-02,0.005970,0.147829,16,11,13.555556,0.222222,...,0.888889,1.0,1.0,0.500000,0.416667,0.416667,0.416667,0.416667,0.416667,0.131988
1,2019-10-02,1,14d,2019-10-02,0.001596,0.016723,16,9,14.200000,0.133333,...,0.933333,1.0,1.0,0.500000,0.444444,0.444444,0.444444,0.444444,0.444444,0.133082
2,2019-10-02,1,21d,2019-10-02,0.001474,0.018557,16,9,14.352941,0.117647,...,0.794118,1.0,1.0,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.131967
3,2019-10-02,1,30d,2019-10-02,0.001206,0.021149,16,5,14.105263,0.157895,...,0.543860,1.0,1.0,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.132030
4,2019-10-02,1,60d,2019-10-02,0.001659,0.028449,16,5,14.615385,0.230769,...,0.506410,1.0,1.0,0.583333,0.500000,0.500000,0.500000,0.500000,0.500000,0.133897
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
765,2019-12-04,15,21d,2019-12-04,0.002998,0.018421,9,4,7.647059,0.352941,...,1.000000,1.0,1.0,0.666667,0.777778,0.833333,0.861111,0.861111,0.861111,0.128039
766,2019-12-04,15,30d,2019-12-04,0.001378,0.020111,9,4,7.500000,0.222222,...,1.000000,1.0,1.0,0.500000,0.666667,0.708333,0.750000,0.750000,0.750000,0.126257
767,2019-12-04,15,60d,2019-12-04,0.004799,0.021744,9,4,7.500000,0.222222,...,1.000000,1.0,1.0,0.500000,0.666667,0.708333,0.750000,0.750000,0.750000,0.126858
768,2019-12-04,15,90d,2019-12-04,0.006008,0.022580,9,4,7.500000,0.222222,...,1.000000,1.0,1.0,0.500000,0.666667,0.708333,0.750000,0.750000,0.750000,0.125623


### Best overall hparams

In [9]:
display_columns = ['time_train', 'avg_recs'] + [ c for c in mdfi.columns if c.endswith('@5') or c.endswith('@10') or c.endswith('@100') ]
overall_hparams = mdfi[mdfi['fold'] > last_folds_idx[0]].groupby(['window_size', 'k']).mean().sort_values(OPTIM_METRIC, ascending=False)
overall_hparams[display_columns]

time_train  avg_recs  precision@5  precision@10  \
window_size k                                                     
7d          7     0.006417  11.94853     0.209294      0.175015   
            5     0.003990  11.94853     0.209294      0.175015   
            15    0.004461  11.94853     0.209294      0.175015   
            9     0.004591  11.94853     0.209294      0.175015   
            10    0.005848  11.94853     0.209294      0.175015   
...                    ...       ...          ...           ...   
90d         8     0.003045  11.72055     0.168887      0.137587   
            4     0.003505  11.72055     0.174037      0.137124   
            5     0.003503  11.72055     0.174037      0.139189   
            15    0.005150  11.72055     0.173934      0.135291   
            1     0.003624  11.72055     0.163480      0.132169   

                precision@100    ndcg@5   ndcg@10  ndcg@100     map@5  \
window_size k                                                           
7d          7        0.018877  0.217189  0.292856  0.304139  0.354374   
            5        0.018877  0.217189  0.292856  0.304139  0.354374   
            15       0.018877  0.217189  0.292856  0.304139  0.354374   
            9        0.018877  0.217189  0.292856  0.304139  0.354374   
            10       0.018877  0.217189  0.292856  0.304139  0.354374   
...                       ...       ...       ...       ...       ...   
90d         8        0.016165  0.312136  0.411696  0.450469  0.280483   
            4        0.016165  0.316344  0.410895  0.450099  0.279909   
            5        0.016165  0.314139  0.412749  0.448078  0.276418   
            15       0.016165  0.310723  0.399247  0.442791  0.275547   
            1        0.016165  0.299985  0.396497  0.444935  0.268029   

                  map@10   map@100  recall@5  recall@10  recall@100  \
window_size k                                                         
7d          7   0.422473  0.432968  0.554392   0.921293         1.0   
            5   0.422473  0.432968  0.554392   0.921293         1.0   
            15  0.422473  0.432968  0.554392   0.921293         1.0   
            9   0.422473  0.432968  0.554392   0.921293         1.0   
            10  0.422473  0.432968  0.554392   0.921293         1.0   
...                  ...       ...       ...        ...         ...   
90d         8   0.339115  0.355109  0.539332   0.857342         1.0   
            4   0.336088  0.352475  0.550216   0.854086         1.0   
            5   0.334626  0.349222  0.550216   0.867188         1.0   
            15  0.328204  0.346159  0.559567   0.841971         1.0   
            1   0.325308  0.345704  0.513367   0.817549         1.0   

                r-precision@5  r-precision@10  r-precision@100  
window_size k                                                   
7d          7        0.617130        0.637037         0.637037  
            5        0.617130        0.637037         0.637037  
            15       0.617130        0.637037         0.637037  
            9        0.617130        0.637037         0.637037  
            10       0.617130        0.637037         0.637037  
...                       ...             ...              ...  
90d         8        0.635039        0.648002         0.648002  
            4        0.611758        0.623884         0.623884  
            5        0.630387        0.641961         0.641961  
            15       0.671884        0.684127         0.684127  
            1        0.645238        0.655600         0.655600  

[77 rows x 17 columns]

Now let's see the behaviour in each fold

In [10]:
best_avg_hparams = mdfi.set_index(['window_size', 'k']).loc[overall_hparams.iloc[0].name].reset_index().set_index(['fold', 'window_size', 'k'])
paths.save_model_results(best_avg_hparams, 'itemknn-best-avg', ORG_NAME, SPLITS_FREQ, SPLITS_NORMALIZE, K_RECOMMENDATIONS)
best_avg_hparams

Saved dataframe into /home/daviddavo/recsys4daos/data/output/Genesis Alpha/models/itemknn-best-avg_7d_normalize.parquet


/tmp/ipykernel_3813312/3951593685.py:1: PerformanceWarning: indexing past lexsort depth may impact performance.
  best_avg_hparams = mdfi.set_index(['window_size', 'k']).loc[overall_hparams.iloc[0].name].reset_index().set_index(['fold', 'window_size', 'k'])


,,,fold_t,time_train,time_rec,open_proposals,min_recs,avg_recs,precision@1,precision@3,precision@5,precision@10,...,recall@10,recall@15,recall@100,r-precision@1,r-precision@3,r-precision@5,r-precision@10,r-precision@15,r-precision@100,time_eval
fold,window_size,k,,,,,,,,,,,,,,,,,,,,,
2019-10-02,7d,7,2019-10-02,0.005685,0.010960,16,11,13.555556,0.222222,0.111111,0.244444,0.155556,...,0.888889,1.000000,1.0,0.500000,0.444444,0.444444,0.444444,0.444444,0.444444,0.128704
2019-10-09,7d,7,2019-10-09,0.016125,0.014352,8,6,7.307692,0.538462,0.282051,0.200000,0.123077,...,1.000000,1.000000,1.0,0.928571,0.875000,0.875000,0.875000,0.875000,0.875000,0.126527
2019-10-16,7d,7,2019-10-16,0.001514,0.007489,8,5,6.833333,0.333333,0.333333,0.266667,0.133333,...,1.000000,1.000000,1.0,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.126303
2019-10-23,7d,7,2019-10-23,0.005697,0.011178,14,7,11.222222,0.111111,0.148148,0.155556,0.166667,...,0.777778,1.000000,1.0,0.250000,0.450000,0.550000,0.550000,0.550000,0.550000,0.127265
2019-10-30,7d,7,2019-10-30,0.014380,0.007725,17,14,16.500000,0.166667,0.222222,0.200000,0.233333,...,0.812500,0.833333,1.0,0.125000,0.437500,0.562500,0.625000,0.625000,0.625000,0.127398
2019-11-06,7d,7,2019-11-06,0.005627,0.013245,17,6,14.272727,0.272727,0.181818,0.127273,0.154545,...,0.977273,1.000000,1.0,0.777778,0.616667,0.616667,0.616667,0.616667,0.616667,0.127268
2019-11-13,7d,7,2019-11-13,0.001723,0.018403,24,19,22.000000,0.294118,0.156863,0.164706,0.158824,...,0.759804,0.926471,1.0,0.450000,0.533333,0.566667,0.600000,0.600000,0.600000,0.131018
2019-11-20,7d,7,2019-11-20,0.005721,0.009976,12,9,10.750000,0.625000,0.416667,0.325000,0.162500,...,1.000000,1.000000,1.0,0.766667,0.833333,0.833333,0.833333,0.833333,0.833333,0.125598
2019-11-27,7d,7,2019-11-27,0.001302,0.015223,12,9,10.428571,0.142857,0.214286,0.200000,0.242857,...,0.964286,1.000000,1.0,0.291667,0.400000,0.466667,0.466667,0.466667,0.466667,0.127111


### Best hparams by fold

This are the parameters used to check which model is the best (validation)

In [11]:
print("Best hyperparams by fold", OPTIM_METRIC)
best_hparams = mdfi.sort_values(OPTIM_METRIC,ascending=False).drop_duplicates(['fold'], keep='first').sort_values('fold').set_index(['fold', 'k', 'window_size'])
paths.save_model_results(best_hparams, 'itemknn-best-val', ORG_NAME, SPLITS_FREQ, SPLITS_NORMALIZE, K_RECOMMENDATIONS)
best_hparams[display_columns]

Best hyperparams by fold map@10
Saved dataframe into /home/daviddavo/recsys4daos/data/output/Genesis Alpha/models/itemknn-best-val_7d_normalize.parquet


,,,time_train,avg_recs,precision@5,precision@10,precision@100,ndcg@5,ndcg@10,ndcg@100,map@5,map@10,map@100,recall@5,recall@10,recall@100,r-precision@5,r-precision@10,r-precision@100
fold,k,window_size,,,,,,,,,,,,,,,,,
2019-10-02,1,7d,0.005970,13.555556,0.244444,0.155556,0.018889,0.111217,0.128723,0.139207,0.307099,0.348765,0.372593,0.722222,0.888889,1.0,0.416667,0.416667,0.416667
2019-10-09,1,14d,0.015011,7.285714,0.214286,0.121429,0.012143,0.363759,0.382446,0.382446,0.728571,0.750680,0.750680,0.892857,1.000000,1.0,0.900000,0.900000,0.900000
2019-10-16,2,21d,0.003186,7.153846,0.246154,0.138462,0.013846,0.353130,0.362412,0.362412,0.716026,0.736630,0.736630,0.961538,1.000000,1.0,0.861111,0.861111,0.861111
2019-10-23,10,30d,0.001374,11.928571,0.185714,0.142857,0.015714,0.228367,0.281804,0.302592,0.350238,0.411406,0.423311,0.596429,0.857143,1.0,0.820000,0.820000,0.820000
2019-10-30,5,30d,0.001546,15.909091,0.200000,0.163636,0.022727,0.254058,0.302970,0.345627,0.376818,0.426488,0.467698,0.503788,0.776515,1.0,0.618056,0.680556,0.680556
2019-11-06,3,7d,0.003659,14.272727,0.127273,0.154545,0.016364,0.299212,0.485562,0.494353,0.273485,0.364646,0.372222,0.356061,0.977273,1.0,0.616667,0.616667,0.616667
2019-11-13,1,7d,0.001390,22.000000,0.211765,0.158824,0.019412,0.269250,0.332158,0.376553,0.348627,0.397397,0.423143,0.495098,0.759804,1.0,0.722222,0.722222,0.722222
2019-11-20,6,7d,0.015053,10.750000,0.325000,0.162500,0.016250,0.497464,0.497464,0.497464,0.725694,0.725694,0.725694,1.000000,1.000000,1.0,0.833333,0.833333,0.833333
2019-11-27,4,10YE,0.034933,9.722222,0.277778,0.211111,0.022222,0.493950,0.606925,0.630607,0.423843,0.509998,0.520099,0.625000,0.916667,1.0,0.628788,0.628788,0.628788


In [12]:
best_hparams.tail(len(best_hparams)-1).describe()

,fold_t,time_train,time_rec,open_proposals,min_recs,avg_recs,precision@1,precision@3,precision@5,precision@10,...,recall@10,recall@15,recall@100,r-precision@1,r-precision@3,r-precision@5,r-precision@10,r-precision@15,r-precision@100,time_eval
count,9,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,...,9.000000,9.000000,9.0,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000
mean,2019-11-06 00:00:00,0.008872,0.017068,13.444444,7.777778,11.852137,0.430969,0.281542,0.226114,0.156910,...,0.920822,0.980046,1.0,0.724239,0.740054,0.759279,0.768869,0.768869,0.768869,0.127686
min,2019-10-09 00:00:00,0.001374,0.009997,8.000000,4.000000,7.153846,0.272727,0.166667,0.127273,0.121429,...,0.759804,0.893939,1.0,0.458333,0.568182,0.616667,0.616667,0.616667,0.616667,0.126125
25%,2019-10-23 00:00:00,0.001546,0.013844,9.000000,5.000000,7.647059,0.352941,0.242424,0.200000,0.142857,...,0.857143,1.000000,1.0,0.666667,0.616667,0.628788,0.680556,0.680556,0.680556,0.126660
50%,2019-11-06 00:00:00,0.003659,0.015465,12.000000,6.000000,10.750000,0.363636,0.309524,0.214286,0.158824,...,0.977273,1.000000,1.0,0.766667,0.780000,0.820000,0.820000,0.820000,0.820000,0.127197
75%,2019-11-20 00:00:00,0.015011,0.018444,17.000000,9.000000,14.272727,0.538462,0.314815,0.247059,0.162500,...,1.000000,1.000000,1.0,0.812500,0.833333,0.833333,0.857143,0.857143,0.857143,0.128042
max,2019-12-04 00:00:00,0.034933,0.033006,24.000000,19.000000,22.000000,0.642857,0.416667,0.325000,0.211111,...,1.000000,1.000000,1.0,0.928571,0.900000,0.900000,0.900000,0.900000,0.900000,0.131054
std,NaN,0.011221,0.006526,5.246692,4.790036,4.894680,0.140148,0.078460,0.056698,0.024556,...,0.099428,0.040422,0.0,0.158631,0.123797,0.113749,0.107837,0.107837,0.107837,0.001488


### Results of using best hparams in next fold

Kind of like the cvtt from [the LightGCN notebook](./11_microsoft_tuning.ipynb).

In [13]:
next_hparams_idx = best_hparams.index.to_frame(False)
next_hparams_idx['fold'] = next_hparams_idx['fold'].shift(-1)
next_hparams_idx = next_hparams_idx.dropna()
assert len(next_hparams_idx) == len(best_hparams)-1
next_hparams = mdfi.set_index(['fold', 'k', 'window_size']).loc[pd.MultiIndex.from_frame(next_hparams_idx)]
paths.save_model_results(next_hparams, 'itemknn-best-test', ORG_NAME, SPLITS_FREQ, SPLITS_NORMALIZE, K_RECOMMENDATIONS)
next_hparams[display_columns]

Saved dataframe into /home/daviddavo/recsys4daos/data/output/Genesis Alpha/models/itemknn-best-test_7d_normalize.parquet


,,,time_train,avg_recs,precision@5,precision@10,precision@100,ndcg@5,ndcg@10,ndcg@100,map@5,map@10,map@100,recall@5,recall@10,recall@100,r-precision@5,r-precision@10,r-precision@100
fold,k,window_size,,,,,,,,,,,,,,,,,
2019-10-09,1,7d,0.010340,7.307692,0.200000,0.123077,0.012308,0.312239,0.338206,0.338206,0.665385,0.695604,0.695604,0.846154,1.000000,1.0,0.875000,0.875000,0.875000
2019-10-16,1,14d,0.007582,7.166667,0.250000,0.141667,0.014167,0.299154,0.308436,0.308436,0.615972,0.638294,0.638294,0.958333,1.000000,1.0,0.708333,0.708333,0.708333
2019-10-23,2,21d,0.001099,11.769231,0.169231,0.146154,0.016154,0.155194,0.223748,0.240720,0.216026,0.296825,0.310229,0.511538,0.884615,1.0,0.450000,0.450000,0.450000
2019-10-30,10,30d,0.005794,15.909091,0.200000,0.163636,0.022727,0.235260,0.284539,0.327196,0.340455,0.391135,0.432344,0.503788,0.776515,1.0,0.641667,0.716667,0.716667
2019-11-06,5,30d,0.001388,13.250000,0.116667,0.091667,0.015833,0.169694,0.255749,0.373933,0.093750,0.127778,0.174581,0.326389,0.597222,1.0,0.250000,0.250000,0.250000
2019-11-13,3,7d,0.000965,22.000000,0.176471,0.158824,0.019412,0.209191,0.296389,0.340785,0.246569,0.321067,0.346812,0.401961,0.759804,1.0,0.566667,0.600000,0.600000
2019-11-20,1,7d,0.003423,10.750000,0.325000,0.162500,0.016250,0.446940,0.446940,0.446940,0.622222,0.622222,0.622222,1.000000,1.000000,1.0,0.708333,0.708333,0.708333
2019-11-27,6,7d,0.014979,10.428571,0.200000,0.242857,0.025000,0.192868,0.368570,0.377572,0.171151,0.311931,0.318424,0.345238,0.964286,1.0,0.466667,0.466667,0.466667
2019-12-04,4,10YE,0.036288,7.500000,0.233333,0.161111,0.016111,0.460250,0.561250,0.561250,0.355185,0.412875,0.412875,0.731481,1.000000,1.0,0.708333,0.750000,0.750000


In [14]:
next_hparams.describe()

,fold_t,time_train,time_rec,open_proposals,min_recs,avg_recs,precision@1,precision@3,precision@5,precision@10,...,recall@10,recall@15,recall@100,r-precision@1,r-precision@3,r-precision@5,r-precision@10,r-precision@15,r-precision@100,time_eval
count,9,9.000000,9.000000,9.000000,9.00000,9.000000,9.000000,9.000000,9.000000,9.000000,...,9.000000,9.000000,9.0,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000
mean,2019-11-06 00:00:00,0.009095,0.016055,13.444444,8.00000,11.786806,0.253713,0.226016,0.207856,0.154610,...,0.886938,0.965385,1.0,0.536817,0.566667,0.597222,0.613889,0.613889,0.613889,0.126447
min,2019-10-09 00:00:00,0.000965,0.010027,8.000000,4.00000,7.166667,0.000000,0.083333,0.116667,0.091667,...,0.597222,0.868056,1.0,0.000000,0.250000,0.250000,0.250000,0.250000,0.250000,0.119748
25%,2019-10-23 00:00:00,0.001388,0.013775,9.000000,4.00000,7.500000,0.142857,0.176471,0.176471,0.141667,...,0.776515,0.926471,1.0,0.291667,0.400000,0.466667,0.466667,0.466667,0.466667,0.125733
50%,2019-11-06 00:00:00,0.005794,0.015209,12.000000,6.00000,10.750000,0.272727,0.212121,0.200000,0.158824,...,0.964286,1.000000,1.0,0.500000,0.616667,0.641667,0.708333,0.708333,0.708333,0.125903
75%,2019-11-20 00:00:00,0.010340,0.015982,17.000000,9.00000,13.250000,0.375000,0.282051,0.233333,0.162500,...,1.000000,1.000000,1.0,0.800000,0.708333,0.708333,0.716667,0.716667,0.716667,0.128397
max,2019-12-04 00:00:00,0.036288,0.027534,24.000000,19.00000,22.000000,0.538462,0.375000,0.325000,0.242857,...,1.000000,1.000000,1.0,0.928571,0.875000,0.875000,0.875000,0.875000,0.875000,0.131568
std,NaN,0.011239,0.004860,5.246692,4.84768,4.823749,0.173144,0.090625,0.058230,0.040599,...,0.145521,0.053946,0.0,0.317834,0.198781,0.185873,0.192435,0.192435,0.192435,0.003209


# User-based KNN

In [15]:
def testHParamsUserKNN(fold, k: int, window_size=None):
    # Create algorithm
    algo = user_knn.UserUser(
        feedback='implicit', # VERY IMPORTANT
        min_sim=0,
        # min_nbrs=0,
        nnbrs=k,  # the maximum number of neighbors for scoring each item (None for unlimited)
    )
    return test_with_hparams_lenskit(algo, all_folds[fold], K_RECOMMENDATIONS, window_size)

pd.Series(testHParamsUserKNN(last_folds_idx[-1],5,'14d'))

fold_t             2019-12-04 00:00:00
time_train                    0.591764
time_rec                      0.842591
open_proposals                       9
min_recs                             5
avg_recs                        8.0625
precision@1                       0.25
precision@3                     0.1875
precision@5                      0.175
precision@10                    0.1625
precision@15                  0.108333
precision@100                  0.01625
ndcg@1                        0.222222
ndcg@3                        0.294751
ndcg@5                        0.347705
ndcg@10                       0.499603
ndcg@15                       0.499603
ndcg@100                      0.499603
map@1                             0.25
map@3                          0.28125
map@5                         0.316771
map@10                        0.409077
map@15                        0.409077
map@100                       0.409077
recall@1                      0.166667
recall@3                 

## Exploring hparams

In [16]:
results = explore_hparams(
    testHParamsUserKNN, 
    ParameterGrid({
        'fold': last_folds_idx,
        'k': ITEMKNN_Ks, # ,20,25,30,35,40,45,50,60,70,80,90,100],
        'window_size': WINDOW_SIZES,
    }),
    paths.hparams_progress('userknn', ORG_NAME, SPLITS_FREQ, SPLITS_NORMALIZE),
)
mdfu = pd.DataFrame(results)
mdfu

Restored checkpoint from ../.cache/Genesis Alpha/hparams-userknn_7d_normalize.pkl with 770 results


  0%|          | 0/770 [00:00<?, ?it/s]

,fold,k,window_size,fold_t,time_train,time_rec,open_proposals,min_recs,avg_recs,precision@1,...,recall@10,recall@15,recall@100,r-precision@1,r-precision@3,r-precision@5,r-precision@10,r-precision@15,r-precision@100,time_eval
0,2019-10-02,1,7d,2019-10-02,0.000390,0.007611,16,11,13.555556,0.222222,...,0.888889,1.0,1.0,0.500000,0.458333,0.458333,0.458333,0.458333,0.458333,0.133233
1,2019-10-02,1,14d,2019-10-02,0.000372,0.012387,16,9,14.200000,0.200000,...,0.933333,1.0,1.0,0.500000,0.433333,0.433333,0.433333,0.433333,0.433333,0.129899
2,2019-10-02,1,21d,2019-10-02,0.000328,0.012537,16,9,14.352941,0.117647,...,0.774510,1.0,1.0,0.500000,0.416667,0.416667,0.416667,0.416667,0.416667,0.130674
3,2019-10-02,1,30d,2019-10-02,0.000398,0.014052,16,5,14.105263,0.157895,...,0.491228,1.0,1.0,0.444444,0.433333,0.433333,0.433333,0.433333,0.433333,0.131285
4,2019-10-02,1,60d,2019-10-02,0.000444,0.019212,16,5,14.615385,0.115385,...,0.423077,1.0,1.0,0.444444,0.428571,0.428571,0.428571,0.428571,0.428571,0.132901
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
765,2019-12-04,15,21d,2019-12-04,0.000367,0.012557,9,4,7.647059,0.117647,...,1.000000,1.0,1.0,0.250000,0.416667,0.500000,0.583333,0.583333,0.583333,0.127450
766,2019-12-04,15,30d,2019-12-04,0.000438,0.013164,9,4,7.500000,0.111111,...,1.000000,1.0,1.0,0.250000,0.583333,0.666667,0.750000,0.750000,0.750000,0.125864
767,2019-12-04,15,60d,2019-12-04,0.000488,0.013613,9,4,7.500000,0.111111,...,1.000000,1.0,1.0,0.250000,0.416667,0.500000,0.583333,0.583333,0.583333,0.125492
768,2019-12-04,15,90d,2019-12-04,0.000556,0.013711,9,4,7.500000,0.111111,...,1.000000,1.0,1.0,0.250000,0.416667,0.500000,0.583333,0.583333,0.583333,0.125463


### Best overall hparams

In [17]:
display_columns = ['time_train', 'avg_recs'] + [ c for c in mdfu.columns if c.endswith('@5') or c.endswith('@10') or c.endswith('@100') ]
overall_hparams = mdfu[mdfu['fold'] > last_folds_idx[0]].groupby(['window_size', 'k']).mean().sort_values(OPTIM_METRIC, ascending=False)
overall_hparams[display_columns]

time_train   avg_recs  precision@5  precision@10  \
window_size k                                                      
10YE        8     0.001032  11.753472     0.188891      0.136773   
            9     0.001014  11.753472     0.188891      0.136773   
            10    0.001034  11.753472     0.188891      0.136773   
            15    0.001014  11.753472     0.188891      0.136773   
            5     0.001012  11.753472     0.188891      0.136773   
...                    ...        ...          ...           ...   
7d          7     0.000341  11.948530     0.231380      0.177260   
            6     0.000327  11.948530     0.231380      0.177260   
            5     0.000326  11.948530     0.231380      0.177260   
            4     0.000334  11.948530     0.231380      0.177260   
            3     0.000333  11.948530     0.230073      0.177260   

                precision@100    ndcg@5   ndcg@10  ndcg@100     map@5  \
window_size k                                                           
10YE        8        0.016047  0.451971  0.527863  0.570453  0.458299   
            9        0.016047  0.451971  0.527863  0.570453  0.458299   
            10       0.016047  0.451971  0.527863  0.570453  0.458299   
            15       0.016047  0.451971  0.527863  0.570453  0.458299   
            5        0.016047  0.450927  0.526818  0.569408  0.457171   
...                       ...       ...       ...       ...       ...   
7d          7        0.018877  0.233527  0.296818  0.306002  0.349889   
            6        0.018877  0.233527  0.296818  0.306002  0.349889   
            5        0.018877  0.233527  0.296818  0.306002  0.349889   
            4        0.018877  0.233231  0.296522  0.305706  0.349290   
            3        0.018877  0.230221  0.295034  0.304218  0.345888   

                  map@10   map@100  recall@5  recall@10  recall@100  \
window_size k                                                         
10YE        8   0.503658  0.520058  0.623193   0.845256         1.0   
            9   0.503658  0.520058  0.623193   0.845256         1.0   
            10  0.503658  0.520058  0.623193   0.845256         1.0   
            15  0.503658  0.520058  0.623193   0.845256         1.0   
            5   0.502531  0.518931  0.623193   0.845256         1.0   
...                  ...       ...       ...        ...         ...   
7d          7   0.411813  0.420516  0.635191   0.936164         1.0   
            6   0.411813  0.420516  0.635191   0.936164         1.0   
            5   0.411813  0.420516  0.635191   0.936164         1.0   
            4   0.411214  0.419917  0.635191   0.936164         1.0   
            3   0.408902  0.417605  0.628655   0.936164         1.0   

                r-precision@5  r-precision@10  r-precision@100  
window_size k                                                   
10YE        8        0.731437        0.745279         0.745279  
            9        0.731437        0.745279         0.745279  
            10       0.731437        0.745279         0.745279  
            15       0.731437        0.745279         0.745279  
            5        0.731437        0.745279         0.745279  
...                       ...             ...              ...  
7d          7        0.608333        0.636111         0.636111  
            6        0.608333        0.636111         0.636111  
            5        0.608333        0.636111         0.636111  
            4        0.608333        0.636111         0.636111  
            3        0.606790        0.636111         0.636111  

[77 rows x 17 columns]

In [18]:
best_avg_hparams = mdfu.set_index(['window_size', 'k']).loc[overall_hparams.iloc[0].name].reset_index().set_index(['fold', 'window_size', 'k'])
paths.save_model_results(best_avg_hparams, 'userknn-best-avg', ORG_NAME, SPLITS_FREQ, SPLITS_NORMALIZE, K_RECOMMENDATIONS)
best_avg_hparams

Saved dataframe into /home/daviddavo/recsys4daos/data/output/Genesis Alpha/models/userknn-best-avg_7d_normalize.parquet


/tmp/ipykernel_3813312/3337470013.py:1: PerformanceWarning: indexing past lexsort depth may impact performance.
  best_avg_hparams = mdfu.set_index(['window_size', 'k']).loc[overall_hparams.iloc[0].name].reset_index().set_index(['fold', 'window_size', 'k'])


,,,fold_t,time_train,time_rec,open_proposals,min_recs,avg_recs,precision@1,precision@3,precision@5,precision@10,...,recall@10,recall@15,recall@100,r-precision@1,r-precision@3,r-precision@5,r-precision@10,r-precision@15,r-precision@100,time_eval
fold,window_size,k,,,,,,,,,,,,,,,,,,,,,
2019-10-02,10YE,8,2019-10-02,0.000890,0.022510,16,5,14.758621,0.241379,0.160920,0.103448,0.072414,...,0.367816,1.000000,1.0,0.452381,0.513889,0.513889,0.513889,0.513889,0.513889,0.133411
2019-10-09,10YE,8,2019-10-09,0.000922,0.018790,8,6,7.500000,0.791667,0.305556,0.200000,0.112500,...,1.000000,1.000000,1.0,0.947368,0.947368,0.947368,0.947368,0.947368,0.947368,0.128558
2019-10-16,10YE,8,2019-10-16,0.000947,0.019126,8,4,7.500000,0.791667,0.375000,0.250000,0.129167,...,1.000000,1.000000,1.0,0.907895,0.925000,0.925000,0.925000,0.925000,0.925000,0.128539
2019-10-23,10YE,8,2019-10-23,0.000963,0.016611,14,6,12.619048,0.571429,0.253968,0.171429,0.128571,...,0.904762,1.000000,1.0,0.937500,0.915385,0.930769,0.930769,0.930769,0.930769,0.130861
2019-10-30,10YE,8,2019-10-30,0.001070,0.014677,17,11,16.333333,0.555556,0.259259,0.200000,0.150000,...,0.863426,0.907407,1.0,0.783333,0.734848,0.757576,0.780303,0.780303,0.780303,0.130594
2019-11-06,10YE,8,2019-11-06,0.001020,0.010795,17,4,13.250000,0.083333,0.194444,0.150000,0.100000,...,0.576389,1.000000,1.0,1.000000,0.583333,0.583333,0.583333,0.583333,0.583333,0.127723
2019-11-13,10YE,8,2019-11-13,0.001068,0.020755,24,12,21.538462,0.038462,0.243590,0.153846,0.107692,...,0.464744,0.634615,1.0,0.500000,0.555556,0.555556,0.574074,0.574074,0.574074,0.134770
2019-11-20,10YE,8,2019-11-20,0.001090,0.009778,12,5,9.818182,0.454545,0.272727,0.163636,0.136364,...,0.909091,1.000000,1.0,0.666667,0.900000,0.900000,0.900000,0.900000,0.900000,0.127827
2019-11-27,10YE,8,2019-11-27,0.001075,0.015077,12,5,9.722222,0.166667,0.148148,0.200000,0.205556,...,0.888889,1.000000,1.0,0.305556,0.416667,0.483333,0.483333,0.483333,0.483333,0.129724


### Best hparams by fold

In [19]:
print("Best hyperparams by fold", OPTIM_METRIC)
best_hparams = mdfu.sort_values(OPTIM_METRIC,ascending=False).drop_duplicates(['fold'], keep='first').sort_values('fold').set_index(['fold', 'k', 'window_size'])
paths.save_model_results(best_hparams, 'userknn-best-val', ORG_NAME, SPLITS_FREQ, SPLITS_NORMALIZE, K_RECOMMENDATIONS)
best_hparams[display_columns]

Best hyperparams by fold map@10
Saved dataframe into /home/daviddavo/recsys4daos/data/output/Genesis Alpha/models/userknn-best-val_7d_normalize.parquet


,,,time_train,avg_recs,precision@5,precision@10,precision@100,ndcg@5,ndcg@10,ndcg@100,map@5,map@10,map@100,recall@5,recall@10,recall@100,r-precision@5,r-precision@10,r-precision@100
fold,k,window_size,,,,,,,,,,,,,,,,,
2019-10-02,1,7d,0.000390,13.555556,0.244444,0.155556,0.018889,0.113549,0.131055,0.141539,0.316049,0.357716,0.381544,0.722222,0.888889,1.0,0.458333,0.458333,0.458333
2019-10-09,8,10YE,0.000922,7.500000,0.200000,0.112500,0.011250,0.682085,0.708052,0.708052,0.822222,0.838591,0.838591,0.916667,1.000000,1.0,0.947368,0.947368,0.947368
2019-10-16,2,21d,0.000374,7.153846,0.246154,0.138462,0.013846,0.414272,0.423555,0.423555,0.891026,0.911630,0.911630,0.961538,1.000000,1.0,0.916667,0.916667,0.916667
2019-10-23,1,30d,0.000409,11.928571,0.214286,0.150000,0.015714,0.346336,0.391962,0.402356,0.613810,0.658353,0.664306,0.682143,0.928571,1.0,0.950000,0.950000,0.950000
2019-10-30,3,10YE,0.000952,16.333333,0.200000,0.150000,0.018889,0.562571,0.630482,0.672824,0.573704,0.613630,0.636489,0.657407,0.863426,1.0,0.777778,0.798611,0.798611
2019-11-06,1,14d,0.000402,13.818182,0.181818,0.154545,0.016364,0.430150,0.565320,0.575591,0.392045,0.472854,0.479847,0.553030,0.969697,1.0,0.716667,0.716667,0.716667
2019-11-13,6,7d,0.000338,22.000000,0.176471,0.158824,0.019412,0.204744,0.277624,0.322020,0.214706,0.283217,0.308962,0.480392,0.759804,1.0,0.458333,0.500000,0.500000
2019-11-20,1,7d,0.000311,10.750000,0.325000,0.162500,0.016250,0.502667,0.502667,0.502667,0.746528,0.746528,0.746528,1.000000,1.000000,1.0,0.809524,0.809524,0.809524
2019-11-27,1,14d,0.000328,9.812500,0.200000,0.225000,0.023125,0.245665,0.436548,0.445550,0.195000,0.333953,0.339635,0.390625,0.968750,1.0,0.416667,0.450000,0.450000


In [20]:
best_hparams.tail(len(best_hparams)-1).describe()[display_columns]

,time_train,avg_recs,precision@5,precision@10,precision@100,ndcg@5,ndcg@10,ndcg@100,map@5,map@10,map@100,recall@5,recall@10,recall@100,r-precision@5,r-precision@10,r-precision@100
count,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.0,9.000000,9.000000,9.000000
mean,0.000571,11.866270,0.215970,0.156994,0.016773,0.427719,0.500824,0.513757,0.533391,0.586039,0.593509,0.707114,0.943361,1.0,0.735334,0.750611,0.750611
min,0.000311,7.153846,0.176471,0.112500,0.011250,0.204744,0.277624,0.322020,0.195000,0.283217,0.308962,0.390625,0.759804,1.0,0.416667,0.450000,0.450000
25%,0.000338,7.500000,0.200000,0.150000,0.015714,0.346336,0.423555,0.423555,0.351481,0.415597,0.415597,0.553030,0.928571,1.0,0.625000,0.666667,0.666667
50%,0.000402,10.750000,0.200000,0.154545,0.016250,0.430150,0.502667,0.502667,0.573704,0.613630,0.636489,0.682143,0.969697,1.0,0.777778,0.798611,0.798611
75%,0.000922,13.818182,0.214286,0.161111,0.018889,0.502667,0.571203,0.575591,0.746528,0.746528,0.746528,0.916667,1.000000,1.0,0.916667,0.916667,0.916667
max,0.001107,22.000000,0.325000,0.225000,0.023125,0.682085,0.708052,0.708052,0.891026,0.911630,0.911630,1.000000,1.000000,1.0,0.950000,0.950000,0.950000
std,0.000322,4.902303,0.045481,0.029796,0.003407,0.149700,0.132590,0.128579,0.258356,0.223229,0.218384,0.215945,0.082516,0.0,0.200633,0.184794,0.184794


### Results of using best hparams in next fold

Kind of like the cvtt from [the LightGCN notebook](./11_microsoft_tuning.ipynb)

In [21]:
next_hparams_idx = best_hparams.index.to_frame(False)
next_hparams_idx['fold'] = next_hparams_idx['fold'].shift(-1)
next_hparams_idx = next_hparams_idx.dropna()
assert len(next_hparams_idx) == len(best_hparams)-1
next_hparams = mdfu.set_index(['fold', 'k', 'window_size']).loc[pd.MultiIndex.from_frame(next_hparams_idx)]
paths.save_model_results(next_hparams, 'userknn-best-test', ORG_NAME, SPLITS_FREQ, SPLITS_NORMALIZE, K_RECOMMENDATIONS)
next_hparams

Saved dataframe into /home/daviddavo/recsys4daos/data/output/Genesis Alpha/models/userknn-best-test_7d_normalize.parquet


,,,fold_t,time_train,time_rec,open_proposals,min_recs,avg_recs,precision@1,precision@3,precision@5,precision@10,...,recall@10,recall@15,recall@100,r-precision@1,r-precision@3,r-precision@5,r-precision@10,r-precision@15,r-precision@100,time_eval
fold,k,window_size,,,,,,,,,,,,,,,,,,,,,
2019-10-09,1,7d,2019-10-09,0.000324,0.009542,8,6,7.307692,0.538462,0.282051,0.200000,0.123077,...,1.000000,1.000000,1.0,0.928571,0.875000,0.875000,0.875000,0.875000,0.875000,0.127057
2019-10-16,8,10YE,2019-10-16,0.000947,0.019126,8,4,7.500000,0.791667,0.375000,0.250000,0.129167,...,1.000000,1.000000,1.0,0.907895,0.925000,0.925000,0.925000,0.925000,0.925000,0.128539
2019-10-23,2,21d,2019-10-23,0.000390,0.009808,14,6,11.769231,0.384615,0.230769,0.215385,0.161538,...,1.000000,1.000000,1.0,0.850000,0.816667,0.925000,0.925000,0.925000,0.925000,0.127920
2019-10-30,1,30d,2019-10-30,0.000427,0.008675,17,11,15.909091,0.545455,0.272727,0.200000,0.154545,...,0.685606,0.893939,1.0,0.659722,0.654762,0.690476,0.708333,0.708333,0.708333,0.129165
2019-11-06,3,10YE,2019-11-06,0.000997,0.010783,17,4,13.250000,0.083333,0.138889,0.150000,0.100000,...,0.576389,1.000000,1.0,1.000000,0.520833,0.583333,0.583333,0.583333,0.583333,0.127075
2019-11-13,1,14d,2019-11-13,0.000410,0.014910,24,14,21.950000,0.150000,0.116667,0.120000,0.085000,...,0.404167,0.700000,1.0,0.611111,0.533333,0.541667,0.569444,0.569444,0.569444,0.132923
2019-11-20,6,7d,2019-11-20,0.000332,0.006343,12,9,10.750000,0.500000,0.416667,0.325000,0.162500,...,1.000000,1.000000,1.0,0.708333,0.694444,0.694444,0.694444,0.694444,0.694444,0.126649
2019-11-27,1,7d,2019-11-27,0.000290,0.010503,12,9,10.428571,0.071429,0.214286,0.214286,0.242857,...,0.964286,1.000000,1.0,0.250000,0.375000,0.472222,0.472222,0.472222,0.472222,0.128461
2019-12-04,1,14d,2019-12-04,0.000357,0.011744,9,5,8.062500,0.187500,0.187500,0.175000,0.162500,...,1.000000,1.000000,1.0,0.611111,0.541667,0.625000,0.666667,0.666667,0.666667,0.129028


In [22]:
next_hparams.describe()

,fold_t,time_train,time_rec,open_proposals,min_recs,avg_recs,precision@1,precision@3,precision@5,precision@10,...,recall@10,recall@15,recall@100,r-precision@1,r-precision@3,r-precision@5,r-precision@10,r-precision@15,r-precision@100,time_eval
count,9,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,...,9.000000,9.000000,9.0,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000
mean,2019-11-06 00:00:00,0.000497,0.011270,13.444444,7.555556,11.880787,0.361384,0.248284,0.205519,0.146798,...,0.847827,0.954882,1.0,0.725194,0.659634,0.703571,0.713272,0.713272,0.713272,0.128535
min,2019-10-09 00:00:00,0.000290,0.006343,8.000000,4.000000,7.307692,0.071429,0.116667,0.120000,0.085000,...,0.404167,0.700000,1.0,0.250000,0.375000,0.472222,0.472222,0.472222,0.472222,0.126649
25%,2019-10-23 00:00:00,0.000332,0.009542,9.000000,5.000000,8.062500,0.150000,0.187500,0.175000,0.123077,...,0.685606,1.000000,1.0,0.611111,0.533333,0.583333,0.583333,0.583333,0.583333,0.127075
50%,2019-11-06 00:00:00,0.000390,0.010503,12.000000,6.000000,10.750000,0.384615,0.230769,0.200000,0.154545,...,1.000000,1.000000,1.0,0.708333,0.654762,0.690476,0.694444,0.694444,0.694444,0.128461
75%,2019-11-20 00:00:00,0.000427,0.011744,17.000000,9.000000,13.250000,0.538462,0.282051,0.215385,0.162500,...,1.000000,1.000000,1.0,0.907895,0.816667,0.875000,0.875000,0.875000,0.875000,0.129028
max,2019-12-04 00:00:00,0.000997,0.019126,24.000000,14.000000,21.950000,0.791667,0.416667,0.325000,0.242857,...,1.000000,1.000000,1.0,1.000000,0.925000,0.925000,0.925000,0.925000,0.925000,0.132923
std,NaN,0.000273,0.003744,5.246692,3.431877,4.711354,0.251605,0.100371,0.058967,0.045935,...,0.230807,0.101814,0.0,0.229741,0.184678,0.168845,0.163569,0.163569,0.163569,0.001876
